# GOALS

* Decide on viewcount threshold to eliminate views?
* Get feature columns working
* Build an LDA model

In [1]:
## testing printing output from console
import subprocess
cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


In [2]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

# Load PySpark and Data Structures

In [3]:
%run -i '1-load-pyspark.py'

## array of dataset names to loop through in analysis
data_array = [
    "english",
    "rus_stackoverflow",
    "superuser",
    "stackoverflow",
    "math"
]

# overwrite to be last one
data_array = [
    "buddhism",
    "economics",
    "fitness",
    "health",
    "interpersonal"
]

datasets = {}   

The Spark UI is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4


# Load Data

In [4]:
%%time
print(datetime.now().time())
#%run -i '2-load-datasets.py'

15:06:58.419304
CPU times: user 373 µs, sys: 121 µs, total: 494 µs
Wall time: 428 µs


In [5]:
def show_spark_df(df, n=5):
    '''
    function to better print spark df entries
    '''
    display(pd.DataFrame(df.head(n), columns=df.columns))

# Clean Data

In [6]:
%%time
print(datetime.now().time())
#%run -i '3-clean-datasets.py'

15:06:58.443372
CPU times: user 281 µs, sys: 80 µs, total: 361 µs
Wall time: 301 µs


# Define Target

In [7]:
%%time
print(datetime.now().time())
#%run -i '4a-best-worst-qs.py'

15:06:58.460841
CPU times: user 142 µs, sys: 53 µs, total: 195 µs
Wall time: 152 µs


# EDA

In [8]:
%%time
print(datetime.now().time())
%run -i '4b-final-eda.py'

15:06:58.470522
Buddhism:


KeyError: 'buddhism'

CPU times: user 12.7 ms, sys: 4.64 ms, total: 17.4 ms
Wall time: 16.6 ms


# Export Clean Data with Target

In [10]:
%%time
print(datetime.now().time())
#%run -i '5-export-data.py'

15:07:10.883463
CPU times: user 120 µs, sys: 88 µs, total: 208 µs
Wall time: 140 µs


In [11]:
for i in data_array:
    s = i.title() + ' & ' + str(round( datasets[i].stat.corr("score", "viewcount"), 2 )) + ' \\\\'
    print(s)

KeyError: 'buddhism'

# Load Cleaned Data

In [12]:
%%time

## load clean data
for i in data_array:
    datasets[i] = (
        spark
        .read
        .load(f'clean-data/{i}.parquet')
)

CPU times: user 3.07 ms, sys: 2.36 ms, total: 5.43 ms
Wall time: 4.22 s


# Train/Test Split Temporally

In [13]:
s = 0
for i in data_array:
    s = s + datasets[i].count()
    print(f'{i}: {datasets[i].count()}')
s

buddhism: 5588
economics: 7380
fitness: 8100
health: 5442
interpersonal: 2962


29472

In [29]:
%%time
print(datetime.now().time())
#%run -i '6a-time-train-test-split-80.py'    
#%run -i '6b-rand-train-test-split-80.py'
#%run -i '6c-time-train-test-split-60.py'
%run -i '6d-rand-train-test-split-60.py'

## check standard deviations of variables
for i in data_array:
    s = i.title() + ' & ' + str(round( pd.to_numeric(train[i].describe('score').select('score').toPandas().iloc[2][0]), 2 )) + ' & ' + \
    str(round( pd.to_numeric(test[i].describe('score').select('score').toPandas().iloc[2][0]), 2 )) + ' \\\\'
    print(s)

16:17:06.385655
Buddhism & 3.6 & 3.63 \\
Economics & 3.62 & 3.07 \\
Fitness & 5.38 & 5.12 \\
Health & 4.11 & 4.07 \\
Interpersonal & 24.95 & 22.78 \\
CPU times: user 87.5 ms, sys: 48.4 ms, total: 136 ms
Wall time: 3 s


In [30]:
'''
interesting to see how skewed rus_stackoverflow posts are to more posts in recent years
'''

'\ninteresting to see how skewed rus_stackoverflow posts are to more posts in recent years\n'

# Create Results Dictionary

In [36]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [32]:
def show_save_results(results, filename='final-results.csv'):
    '''
    function to print and export modelling results
    '''
    display(pd.DataFrame.from_dict(results).T)
    print(pd.DataFrame.from_dict(results).T.to_latex())
    pd.DataFrame.from_dict(results).T.to_csv(filename)

# Silly Mean Model

In [33]:
from pyspark.sql.functions import array, lit, struct

## choose target variable
target = 'score'

## create mean dictionaries
y_ravi_tr_means = {}

## calculate the mean of each forum, using ONLY training set
for i in data_array:
    y_ravi_tr_means[i] = train[i].select(target).rdd.flatMap(lambda x: x).mean()

## import rmse evaluator
from pyspark.ml.evaluation import RegressionEvaluator

## create dictionaries for training and testing (baseline) rmse 
base = {}
tr_rmse = {}

## modelling
for i in data_array:

    ## initial variable for timing
    t0 = time.time()
    
    ## train silly mean model by assigning training set mean for training and testing predictions
    train[i] = train[i].withColumn('mean_pred', lit(y_ravi_tr_means[i]))
    test[i] = test[i].withColumn('mean_pred', lit(y_ravi_tr_means[i]))

    ## evaluate silly mean model, on both training and testing set
    evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='mean_pred')
    tr_rmse[i] = round( evaluator.evaluate(train[i]), 2)
    base[i] = round( evaluator.evaluate(test[i]), 2)

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {base[i]}")

    ## record time taken
    timet = round( time.time() - t0, 2 )
    
    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i.title()]['0silly_mean.0tr_rmse'] = tr_rmse[i]
    RESULTS[i.title()]['0silly_mean.1rmse'] = base[i]
    RESULTS[i.title()]['0silly_mean.2timet'] = timet
    
## record results
show_save_results(RESULTS)

The root-mean-square error of buddhism's mean model is 3.63
The root-mean-square error of economics's mean model is 3.07
The root-mean-square error of fitness's mean model is 5.12
The root-mean-square error of health's mean model is 4.08
The root-mean-square error of interpersonal's mean model is 22.77


,0silly_mean.0tr_rmse,0silly_mean.1rmse,0silly_mean.2timet
Buddhism,3.60,3.63,0.37
Economics,3.62,3.07,0.29
Fitness,5.38,5.12,0.26
Health,4.11,4.08,0.26
Interpersonal,24.94,22.77,0.25


\begin{tabular}{lrrr}
\toprule
{} &  0silly\_mean.0tr\_rmse &  0silly\_mean.1rmse &  0silly\_mean.2timet \\
\midrule
Buddhism      &                  3.60 &               3.63 &                0.37 \\
Economics     &                  3.62 &               3.07 &                0.29 \\
Fitness       &                  5.38 &               5.12 &                0.26 \\
Health        &                  4.11 &               4.08 &                0.26 \\
Interpersonal &                 24.94 &              22.77 &                0.25 \\
\bottomrule
\end{tabular}



# Viewcount Model

In [35]:
%%time
print(datetime.now().time())
#3min 56s

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['viewcount']
datet_variables = ['clean_date']

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100, # this doesn't change anything
                      #regParam=0.3, # using regularisation parameter here useless since there is one feature
                      #elasticNetParam=0.8,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='viewcount_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
#.addGrid(lr.regParam, [1e-3, 1.])
#.addGrid(lr.elasticNetParam, [1e-3, 1.])
paramGrid = ParamGridBuilder() \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='viewcount_pred')

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    
    ## fitting on train and predicting on train/test
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['1viewcount.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['1viewcount.1rmse'] = rmse
    RESULTS[i.title()]['1viewcount.2imprv'] = impr
    RESULTS[i.title()]['1viewcount.3timet'] = timet
    
## record results
show_save_results(RESULTS)

16:17:45.907751
The root-mean-square error of buddhism's viewcount model is 3.24
The root-mean-square error of economics's viewcount model is 2.95
The root-mean-square error of fitness's viewcount model is 4.73
The root-mean-square error of health's viewcount model is 4.03
The root-mean-square error of interpersonal's viewcount model is 10.38


,1viewcount.0tr_rmse,1viewcount.1rmse,1viewcount.2imprv,1viewcount.3timet
Buddhism,3.24,3.24,10.74,3.51
Economics,3.45,2.95,3.91,3.30
Fitness,5.07,4.73,7.62,3.00
Health,4.08,4.03,1.23,2.69
Interpersonal,12.07,10.38,54.41,2.76


\begin{tabular}{lrrrr}
\toprule
{} &  1viewcount.0tr\_rmse &  1viewcount.1rmse &  1viewcount.2imprv &  1viewcount.3timet \\
\midrule
Buddhism      &                 3.24 &              3.24 &              10.74 &               3.51 \\
Economics     &                 3.45 &              2.95 &               3.91 &               3.30 \\
Fitness       &                 5.07 &              4.73 &               7.62 &               3.00 \\
Health        &                 4.08 &              4.03 &               1.23 &               2.69 \\
Interpersonal &                12.07 &             10.38 &              54.41 &               2.76 \\
\bottomrule
\end{tabular}

CPU times: user 959 ms, sys: 277 ms, total: 1.24 s
Wall time: 15.3 s


In [ ]:
'''
Interesting that there are different improvements of viewcount over mean-only prediciton
'''

# Begin Feature Engineering

In [25]:
## garbage collector to speed up computation
import gc
collected = gc.collect()
print('Garbage collector: collected %d objects.' % collected)

Garbage collector: collected 1559 objects.


In [26]:
#######################
###### TOKENISER ######
#######################

import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation
class NLTKWordPunctTokeniser(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokeniser, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            # get tokens with separate punctuation
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            # sort out russian stopwords
            #if ('я' in tokens):
            #    tokens = [t for t in tokens if t not in nltk.corpus.stopwords.words('russian')]
            # remove english stopwords
            tokens = [t for t in tokens if t not in stopwords]
            # remove single letters
            #tokens = [t for t in tokens if not len(t)==1] # this removes single punctuation as well
            # stemming the words
            tokens = [ps.stem(t) for t in tokens]
            # convert to lower case
            return [t.lower() for t in tokens]
    
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [37]:
%%time
print(datetime.now().time())
# ENGLISH
# 8min 56s for no CV and no GRIDSEARCH
# 17min 10s for 3-CV and no GRIDSEARCH
# SMALL DATASETS
# 36min 55s for no CV and no GRIDSEARCH
# 12min 35s for 3-CV and no GRIDSEARCH
# 8min 32s for 2-CV and no GRIDSEARCH

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
textt_variables = ['title', 'clean_body']
datet_variables = ['clean_date']

## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

## textual columns
# tokenising text cols with custom transformer
nltk_tokeniser_body = NLTKWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokeniser_title = NLTKWordPunctTokeniser(
    inputCol='title', outputCol='titl_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

# count occurence of tokens, i.e. create dfm
cnt_vectrizr_body = CountVectorizer(inputCol='body_words', outputCol='body_feats', minDF=2)
cnt_vectrizr_title = CountVectorizer(inputCol='titl_words', outputCol='titl_feats', minDF=2)

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['body_feats', 'titl_feats'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    nltk_tokeniser_body, 
    nltk_tokeniser_title,
    #token_counter_body,
    #token_counter_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(maxIter=100,
                      regParam=1,
                      elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='tokens_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
#    .addGrid(lr.elasticNetParam, [1e-3, 1.])
#    .addGrid(lr.regParam, [1e-3, 1.])
paramGrid = ParamGridBuilder() \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='tokens_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m tokens\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2tokens.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2tokens.1rmse'] = rmse
    RESULTS[i.title()]['2tokens.2imprv'] = impr
    RESULTS[i.title()]['2tokens.3timet'] = timet
    RESULTS[i.title()]['2tokens.4elastic'] = ela_param
    RESULTS[i.title()]['2tokens.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

16:18:29.791803
The root-mean-square error of buddhism's tokens model is 3.63
The root-mean-square error of economics's tokens model is 3.07
The root-mean-square error of fitness's tokens model is 5.12
The root-mean-square error of health's tokens model is 4.08
The root-mean-square error of interpersonal's tokens model is 23.15


,2tokens.0tr_rmse,2tokens.1rmse,2tokens.2imprv,2tokens.3timet,2tokens.4elastic,2tokens.5regular
Buddhism,3.60,3.63,-0.00,240.07,1.0,1.0
Economics,3.61,3.07,-0.00,331.80,1.0,1.0
Fitness,5.38,5.12,-0.00,421.14,1.0,1.0
Health,4.11,4.08,-0.00,189.87,1.0,1.0
Interpersonal,18.30,23.15,-1.67,219.58,1.0,1.0


\begin{tabular}{lrrrrrr}
\toprule
{} &  2tokens.0tr\_rmse &  2tokens.1rmse &  2tokens.2imprv &  2tokens.3timet &  2tokens.4elastic &  2tokens.5regular \\
\midrule
Buddhism      &              3.60 &           3.63 &           -0.00 &          240.07 &               1.0 &               1.0 \\
Economics     &              3.61 &           3.07 &           -0.00 &          331.80 &               1.0 &               1.0 \\
Fitness       &              5.38 &           5.12 &           -0.00 &          421.14 &               1.0 &               1.0 \\
Health        &              4.11 &           4.08 &           -0.00 &          189.87 &               1.0 &               1.0 \\
Interpersonal &             18.30 &          23.15 &           -1.67 &          219.58 &               1.0 &               1.0 \\
\bottomrule
\end{tabular}

CPU times: user 8.26 s, sys: 2.25 s, total: 10.5 s
Wall time: 23min 22s


In [ ]:
## check predictions aren't constant
models['health'].transform(test['health']).select('tokens_pred').take(10)

In [ ]:
"""why the heck does everything besides interpersonal have constant predictions - it's not the parameters or the size of the data"""
"""it's the size of the datasets"""

In [ ]:
## have a look at CV models params
list(zip(models['health'].avgMetrics, paramGrid))

In [ ]:
## extract best parameters
for i in data_array:
    # elasticnet
    ela_key = list(models[i].bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = models[i].bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(models[i].bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = models[i].bestModel.stages[-1].extractParamMap()[reg_key]
    print(i)
    print(f'elastic net: {ela_param}, reg: {reg_param}')

# Trying to get sentence and token count to work

In [ ]:
df_ = spark.createDataFrame(datasets['economics'].take(5))

In [ ]:
show_spark_df(df_)

In [ ]:
word_feat_list = tester4.transform( tester3.fit(tester.transform(df_)).transform(tester.transform(df_)) )

In [ ]:
word_feat_list

In [ ]:
word_feat_list.select('features').collect()

In [ ]:
show_spark_df(word_feat_list)

In [ ]:
from pyspark.mllib.linalg import Vectors
def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

In [ ]:
word_feat_list.select("features").rdd.map(lambda r: r[0][0]).collect()

In [ ]:
word_feat_list.select("features").rdd.map(lambda r: r[0]).map(lambda r: (r,1)).reduceByKey(lambda a,b: a+b).collect()

In [ ]:
tester = NLTKWordPunctTokeniser(
    inputCol='title', outputCol='title_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

tester2 = NLTKWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

In [ ]:
tester3 = CountVectorizer(inputCol="title_words", outputCol="features")

In [ ]:
tester4 = NLTKCounter(inputCol='features', outputCol='final_features')

In [ ]:
########################
###### SENTENISER ######
########################

import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation
class NLTKSenteniser(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokenizer, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            # get tokens with separate punctuation
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            # sort out russian stopwords
            if ('я' in tokens):
                tokens = [t for t in tokens if t not in nltk.corpus.stopwords.words('russian')]
            # remove english stopwords
            tokens = [t for t in tokens if t not in stopwords]
            # remove single letters
            tokens = [t for t in tokens if not len(t)==1]
            # stemming the words
            tokens = [ps.stem(t) for t in tokens]
            # convert to lower case
            return [t.lower() for t in tokens]
    
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
#######################
####### COUNTER #######
#######################

import nltk
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation
class NLTKCounter(Transformer, HasInputCol, HasOutputCol):
    '''
    Takes in cnt_vectrizr to count number of tokens per question
    '''

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(NLTKCounter, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, dataset):

        def f(s):
            return 1
    
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
a

In [ ]:
## import elements from natural language toolkit
import nltk
#nltk.download('all') # uncomment after first run as admin check
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
lmtzr = WordNetLemmatizer()

def get_tokens(line):
    '''
    Function to parse text features
    '''
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuations from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stopwords
    words = [w for w in words if not w in stop_words]
    # lemmatizing the words, see https://en.wikipedia.org/wiki/Lemmatisation
    '''lemmatise or stem???'''
    words = [lmtzr.lemmatize(w) for w in words]
    # remove single letters
    words = [word for word in words if not len(word)==1]
    return (words)

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import Bucketizer
from pyspark.sql import DataFrame
from typing import Iterable
import pandas as pd

## custom transformer to spread sparse vectors into individual columns
class VectorMLliber(Transformer):
    """
    A custom Transformer which converts a column of pyspark.ml vectors to multiple pyspark.mllib vectors.
    """

    def __init__(self, inputCol=None):
        super(VectorMLliber, self).__init__()

    def _transform(self, df: DataFrame) -> DataFrame:
        
        def f(v):
            return Vectors.sparse(v.size, v.indices, v.values)
        
        df = df.rdd.map(lambda r: as_mllib_vector(r[0]))
        return df

In [ ]:
# ???????????
VectorMLliber_body = VectorMLliber(inputCol='body_features')
VectorMLliber_title = VectorMLliber(inputCol='titl_features')

In [ ]:
'''def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

features = {}
feature_vec_list = {}
for i in data_array:
    features[i] = word_feat_list[i].select("features")
    feature_vec_list[i] = features[i].rdd.map(lambda r: as_mllib_vector(r[0]))
    feature_vec_list[i].cache()
'''

# Save predictions

In [ ]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

#Now we can predict directly!

reloaded.predict(X)[:10]

In [ ]:
## save models DOESN'T WORK BECAUSE: 'NLTKWordPunctTokenizer' object has no attribute '_to_java'
for i in data_array:
    param_dict[i].save(f'{i}-pipeline') 

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb